In [1]:
import requests

In [4]:
url = "https://gist.githubusercontent.com/pxsdirac/940436aa85580ac3d72a8cce8cbcc022/raw/aeb7da0710bb87b5106b7b564f40ac4c538fc9a5/movie.csv"
resp = requests.get(url)
resp

<Response [200]>

In [14]:
from collections import namedtuple


In [15]:
lines = resp.content.decode('utf8').split('\n')
header = lines[0].split(',')
Movie = namedtuple('Movie', header)

In [58]:
import sys
class Movie:
    def __init__(self, name, running_time, rating, title):
        try:
            self.name = str(name)
            self.running_time = float(running_time)
            self.rating = float(rating)
            self.title = str(title)
        except:
            print(f"invalid row: {name}, {running_time}, {rating}, {title}", file=sys.stderr)
    
    def __lt__(self, another) -> bool:
        return self.rating > another.rating

    def __repr__(self) -> str:
        return f"name: {self.name}, title: {self.title}, rating: {self.rating}"

In [17]:
header

['name', 'running_time', 'rating', 'title']

In [2]:
from typing import List 
def parse_csv(line:str) -> List[str]:
    dp = line.find('"')
    if dp == -1:
        return line.split(',')
    else:
        ret = []
        dp = -1
        s, i = 0, 0
        while i < len(line):
            c = line[i]
            # print(f"{i}")
            if c == '"' and dp == -1: # find first "
                dp = i
                # print(f"first {dp}")
                i += 1
                continue
            if dp >= 0 and c == '"':
                # print(f"{dp}, {i}")
                ret.append(line[dp+1:i])
                dp = -1
                i += 1
                if i < len(line):
                    if line[i] == ',':
                        i += 1
                s = i
                continue
            if c == ',':
                if dp >= 0: # inside one double quote
                    i += 1
                    continue
                
                # print(f"first norm field: {s}, {i}")
                ret.append(line[s:i])
                s = i+1
                
            i += 1
        if s < len(line):
            ret.append(line[s:])
    return ret

In [3]:
print(parse_csv("a,b,c,d"))
print(parse_csv('"27, most popular",127.6053254325764,6.03051434574081,title of movie27'))
print(parse_csv('26,74.64905326075485,1.6806938100374067,"The HD-remastered classic 60s crime universe of Gotham Citys fiercest fighter, Batman, comes to life with Robin, Catwoman, the Joker and more."'))

['a', 'b', 'c', 'd']
['27, most popular', '127.6053254325764', '6.03051434574081', 'title of movie27']
['26', '74.64905326075485', '1.6806938100374067', 'The HD-remastered classic 60s crime universe of Gotham Citys fiercest fighter, Batman, comes to life with Robin, Catwoman, the Joker and more.']


In [59]:
movie_rows = []
col_num = len(header)
for line in lines[1:]:
    col = parse_csv(line)
    if len(col) == col_num:
        mv = Movie(*col)
        movie_rows.append(mv)
    else:
        print(f"invalid line: {line}")
    

invalid line: "27, most popular",127.6053254325764,6.03051434574081,title of movie27


In [60]:
sorted(movie_rows)[:10]

[name: 111, title: title of movie111, rating: 10.0,
 name: 113, title: title of movie113, rating: 10.0,
 name: 152, title: title of movie152, rating: 10.0,
 name: 187, title: title of movie187, rating: 10.0,
 name: 216, title: title of movie216, rating: 10.0,
 name: 239, title: title of movie239, rating: 10.0,
 name: 253, title: title of movie253, rating: 10.0,
 name: 286, title: title of movie286, rating: 10.0,
 name: 519, title: title of movie519, rating: 10.0,
 name: 520, title: title of movie520, rating: 10.0]